In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'videos_CNN_large'

In [3]:
config_path = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME}.yaml")
config = load_config(config_path)

In [4]:
model = CondRealNVP.from_config(config).to(device)

print(f'{model.n_params:,}')

67,691,783


In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 1/1 [00:02<00:00,  2.79s/it, file=fixed_data_render_2s_15FPS_1.pkl]


Using videos data for training. Shapes:
X shape: torch.Size([1000, 2, 30, 90, 160])
y shape: torch.Size([1000, 19])


In [6]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Train: 4.8742 - Val: 3.6904 (avg: 7.2452, min: 7.3677) | lr: 2.00e-04 - Patience: 1/500 - z: (0.1039 ± 0.4361) ± (1.0018 ± 0.3354):   0%|          | 30/50000 [01:46<49:22:25,  3.56s/it]   
Traceback (most recent call last):
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 100, in train
    model = self._train(
            ^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 227, in _train
    loss = self._train_batch(x, y, model, optimizer, loss_function)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 310, in _train_batch
    loss.backward()
  File "/home/psaegert/miniconda3/envs/bcnf/lib/python3.11/site-packages/torch/_tensor.py", line 522, in backward
    torch.autograd.backward(
  File "/home/psaegert/miniconda3/envs/bcnf/lib/python3.11/site-packages/torch/autograd/__init__.py", line 266, in backward
    Variable._execution_engine.run_backward

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


distance_to_last_best_val_loss_fold_-1,▁█████████████████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
lr_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time_fold_-1,▁▁▁▁▁▁████████████████████████
train_loss_fold_-1,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_fold_-1,█▆▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
z_mean_mean_fold_-1,▄▇▅▆▆▄▅▁▁▆▅▄▅▃▂▅▄▅▄▄▆▄▅▇█▆▅█▄▆
z_mean_std_fold_-1,▄▆▁▂▅▂▄▄█▄▁▇▄▂▄▂▄▁▄▃▆▂▅▅▇▄▃▄▄▅
z_std_mean_fold_-1,▁▄▆▄▃▄▅▅▃▄▅▃▆▃▅█▅▁▃▃▅▁▄▇▅▅▅▅▅▇
z_std_std_fold_-1,▁▂█▇▃▂▂▂▆▄▃▄▂▁▆▂▅▅█▂▆▂▆▆▄▆█▃▁▇
distance_to_last_best_val_loss_fold_-1,1


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/trajectory_FC_small
